In [2]:
# SST inspector (GeoTIFF): abre 1 SKT.tif y muestra metadatos + (opcional) DataArray
# Requisitos: rasterio  (opcional: rioxarray)

from pathlib import Path
import importlib
import re
import numpy as np

# ---------- localizar carpeta ----------
def find_sst_sample_dir(start: Path | None = None) -> Path:
    start = (start or Path.cwd()).resolve()
    for parent in [start, *start.parents]:
        cand = parent / "downloads" / "sst" / "sample"
        if cand.is_dir():
            return cand
    raise FileNotFoundError(f"No se encontró 'downloads/sst/sample' desde {start}")

sample_dir = find_sst_sample_dir()
tifs = sorted(sample_dir.glob("*.tif"))
assert tifs, f"No hay .tif en {sample_dir}"

# prioriza SKT.tif; si no, toma el primero
skt_files = [p for p in tifs if p.name.endswith("_SKT.tif")]
tif_path = skt_files[0] if skt_files else tifs[0]

print(f"📂 Carpeta: {sample_dir}")
print("🗂️  Archivos (primeros 8):")
for p in (skt_files or tifs)[:8]:
    print("   •", p.name)
print(f"\n📄 Abriendo: {tif_path.name}")

# ---------- abrir con rasterio ----------
if importlib.util.find_spec("rasterio") is None:
    raise RuntimeError(
        "Falta rasterio. Instala con:  python -m pip install rasterio\n"
        "(Opcional) Para ver como xarray: python -m pip install rioxarray"
    )

import rasterio
from rasterio.windows import Window

with rasterio.open(tif_path) as src:
    print("\n— Raster info —")
    print("driver:", src.driver)
    print("bands:", src.count)
    print("size (width x height):", src.width, "x", src.height)
    print("dtype:", src.dtypes[0] if src.count else None)
    print("crs:", src.crs)
    print("transform (affine):", src.transform)
    try:
        print("resolution:", src.res)
    except Exception:
        pass
    print("bounds:", src.bounds)
    print("nodata:", src.nodatavals[0] if src.count else None)

    # tags generales y de banda 1 (si existen)
    try:
        ds_tags = src.tags()
        if ds_tags:
            print("\n— Dataset tags (claves):", list(ds_tags.keys())[:20])
            # imprime algunas claves típicas si están
            for k in ("units", "scale_factor", "add_offset", "long_name"):
                if k in ds_tags:
                    print(f"   {k}: {ds_tags[k]}")
    except Exception:
        pass

    try:
        b1_tags = src.tags(1)
        if b1_tags:
            print("\n— Band 1 tags (claves):", list(b1_tags.keys())[:20])
            for k in ("units", "scale_factor", "add_offset", "long_name"):
                if k in b1_tags:
                    print(f"   {k}: {b1_tags[k]}")
    except Exception:
        pass

    # pequeña muestra para stats rápidos (evita cargar todo si es grande)
    try:
        win = Window(0, 0, min(512, src.width), min(512, src.height))
        arr = src.read(1, window=win, masked=True)
        finite = np.asarray(arr).astype("float64")
        m = np.isfinite(finite)
        if m.any():
            vmin = float(np.nanmin(finite[m]))
            vmax = float(np.nanmax(finite[m]))
            print("\n— Stats (ventana 512x512 desde esquina):")
            print("   min:", vmin, "max:", vmax)
    except Exception as e:
        print("\n(Stats sample falló)", e)

# ---------- vista xarray opcional ----------
if importlib.util.find_spec("rioxarray") is not None:
    import rioxarray as rxr
    da = rxr.open_rasterio(tif_path, masked=True)  # dims: band, y, x
    print("\n✅ rioxarray DataArray:")
    display(da)

    print("\n— Resumen compacto —")
    try:
        # dims y tamaños
        print("dims & sizes:", dict(da.sizes))
        # coord keys
        print("coords:", list(da.coords))
        # attrs (claves)
        print("attrs keys:", list(da.attrs.keys())[:20])
        # info espacial
        if hasattr(da, "rio"):
            print("CRS:", da.rio.crs)
            try:
                print("Bounds:", da.rio.bounds())
            except Exception as e:
                print("Bounds: (no disponibles)", e)
    except Exception as e:
        print("(resumen xarray falló)", e)
else:
    print("\nℹ️ (Opcional) Instala rioxarray para ver como xarray:  python -m pip install rioxarray")


📂 Carpeta: C:\Users\Crist\Desktop\NASA\tag-and-satellite-data-model\downloads\sst\sample
🗂️  Archivos (primeros 8):
   • AST_08_00401172010155808_20250702031314_SKT.tif
   • AST_08_00401172010155817_20250702031343_SKT.tif
   • AST_08_00401172010155826_20250702031700_SKT.tif
   • AST_08_00401172010155835_20250702031723_SKT.tif
   • AST_08_00401172010155843_20250702031428_SKT.tif
   • AST_08_00401172010155852_20250702031634_SKT.tif
   • AST_08_00401172010155901_20250702031600_SKT.tif
   • AST_08_00404162010155216_20250703172659_SKT.tif

📄 Abriendo: AST_08_00401172010155808_20250702031314_SKT.tif

— Raster info —
driver: GTiff
bands: 1
size (width x height): 830 x 700
dtype: uint16
crs: EPSG:32617
transform (affine): | 89.08,-12.85, 559035.98|
|-12.85,-89.08, 1696892.96|
| 0.00, 0.00, 1.00|
resolution: (90.0, 90.0)
bounds: BoundingBox(left=550044.2536483303, bottom=1623876.3129981384, right=632971.2220980669, top=1696892.9552079116)
nodata: None

— Dataset tags (claves): ['OperationalQual

<xarray.DataArray (band: 1, y: 700, x: 830)> Size: 2MB
[581000 values with dtype=float32]
Coordinates:
  * band         (band) int64 8B 1
    xc           (y, x) float64 5MB 5.591e+05 5.592e+05 ... 6.239e+05 6.239e+05
    yc           (y, x) float64 5MB 1.697e+06 1.697e+06 ... 1.624e+06 1.624e+06
    spatial_ref  int64 8B 0
Dimensions without coordinates: y, x
Attributes:
    OperationalQualityFlagExplanation:  All requested inputs were used
    AREA_OR_POINT:                      Area
    scale_factor:                       1.0
    add_offset:                         0.0


— Resumen compacto —
dims & sizes: {'band': 1, 'y': 700, 'x': 830}
coords: ['band', 'xc', 'yc', 'spatial_ref']
attrs keys: ['OperationalQualityFlagExplanation', 'AREA_OR_POINT', 'scale_factor', 'add_offset']
CRS: EPSG:32617
Bounds: (559035.9810666677, 1696892.9552079116, 633735.9810666677, 1759892.9552079116)
